In [5]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

from sklearn.model_selection import cross_val_score, train_test_split

%matplotlib inline
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [6]:
#Divede the train, validation and test dataset.
file_handler = open("../pairsDevTrain.txt", "r", encoding="utf-8")

# We must first put the data in a list
train_same = [];
train_diff= [];

for row in file_handler:
    # The values we read are strings, we must convert them to the correct type (float)
    temp=row.replace("\n", "");
    string= temp.split("\t") # split it by whitespace
    converted = [x for x in string]
    if len(converted) ==3:
        #train_same.append( [converted[0],converted[1],converted[0],converted[2] ]);
        train_same.append( converted );
    if len(converted) ==4:
        train_diff.append(converted);
file_handler.close()


file_handler = open("../pairsDevTest.txt", "r", encoding="utf-8")
# We must first put the data in a list
test_same = [];
test_diff= [];

for row in file_handler:
    temp=row.replace("\n", "");
    string= temp.split("\t") # split it by whitespace
    converted = [x for x in string]
    if len(converted) ==3:
        test_same.append( converted );
    if len(converted) ==4:
        test_diff.append(converted);
file_handler.close()

print("There are {} images in the train, {} in the test".format(len(train_same)+ len(train_diff), len(test_same)+ len(test_diff) ) );


vali_same= train_same[0: 250: 1];
vali_diff= train_diff[0: 250: 1];

train_same=train_same[250: : 1]
train_diff=train_diff[250: : 1]

There are 2200 images in the train, 1000 in the test


In [7]:
len( train_same), len(train_diff), len(vali_same), len(vali_diff), len(test_same), len(test_diff)

(850, 850, 250, 250, 500, 500)

In [10]:
width, height, channels=96, 96, 3
def format_filename(name, number):
    num_zeros = "0"*(4 - len(number))
    filepath = "../lfw-deepfunneled/"+name+"/"+name+"_"+num_zeros+number+".jpg"
    return filepath


def extractFaceCo (imagePath): 
    cascPath = "haarcascade_frontalface_default.xml"
    # Create the haar cascade
    faceCascade = cv2.CascadeClassifier(cascPath)

    # Read the image
    image = cv2.imread(imagePath ) #read a RGB image
    #print(image.shape)
    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        image,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    #flags = cv2.CV_HAAR_SCALE_IMAGE
    )
     
    # Draw a rectangle around the faces
    if len(faces)==0:  #no face is detected
        print(imagePath)
        return np.array([])
    
    best_x, best_y, best_w, best_h=0,0,250,250;
    best_size= -np.float("inf")
    for (x, y, w, h) in faces: #There might be more than one face
        if w*h> best_size:
            best_size= w*h
            best_x, best_y, best_w, best_h= x, y, w, h
    
    
    image1= image[ best_y:best_y+best_h, best_x:best_x+best_w ]
    face= cv2.resize(image1, (width, height), interpolation = cv2.INTER_AREA)
    
    #print(face.shape)
    #print(face[0, 0:5])
    return face #is returning a local variable legal? 


def get_batch(batch_size, same, diff):
    # initialize 2 empty arrays for the input image batch

    pairs=[np.zeros( (batch_size, height, width, channels) ) for i in range(2)]    
    # initialize vector for the targets
    targets=np.zeros( batch_size )
    
    num_fails=0;
    not_enough=0
    for i in range( batch_size//2 ):
        
        img1= extractFaceCo(format_filename( same[i][0], same[i][1] )  )
        img2= extractFaceCo(format_filename( same[i][0], same[i][2] )  )        
        while (len(img1)==0 or len(img2)==0): #fails to detect the face
            num_fails+=1
            if (i+ num_fails >= len(same) ):
                print("Did not find enough qualified images, please reduce the batch_size")
                not_enough=1;
                break;
            
            img1= extractFaceCo(format_filename( same[-num_fails][0], same[-num_fails][1] )  )
            img2= extractFaceCo(format_filename( same[-num_fails][0], same[-num_fails][2] )  )
        
        if not_enough:
            break;
        pairs[0][i, :, :, :]= img1.reshape(height, width, -1)
        pairs[1][i, :, :, :]= img2.reshape(height, width, -1)   
        targets[i]= 1;
        
        #print to check whether it is correct
        #print("now in get_batch")
        #print(img1[0, 0:5])
        #print( pairs[0][i, 0, 0:5, 0])
        
    num_fails-= num_fails;  
    not_enough=0
    for i in range( batch_size//2, batch_size ):
        j= i- batch_size//2; #j start from 0
        img1= extractFaceCo(format_filename( diff[j][0], diff[j][1] )  )
        img2= extractFaceCo(format_filename( diff[j][2], diff[j][3] )  )
            
        while (len(img1)==0 or len(img2)==0): #fails to detect the face
            num_fails+=1
            if (j+ num_fails >= len(diff) ): #i+num_fails== len(diff): they are pointing to the same element.
                print("Did not find enough qualified images, please reduce the batch_size")
                not_enough=1;
                break;
            img1= extractFaceCo(format_filename( diff[-num_fails][0], diff[-num_fails][1] )  )
            img2= extractFaceCo(format_filename( diff[-num_fails][2], diff[-num_fails][3] )  )
        
        if not_enough:
            break;    
        pairs[0][i, :, :, :]= img1
        pairs[1][i, :, :, :]= img2
        targets[i]= 0;  
        
        #print to check whether it is correct
        #print("now in get_batch")
        #print(img2[0, 0:5])
        #print( pairs[1][i, 0, 0:5, 0])
    return pairs, targets

def fillIdent(batch_size, same, diff):       
    name={}
    ident_in=[];
    indet_out=[];
    ident_y=[]
    
    i=0    
    num_fails=0;
    num_same=0;
    not_enough=0
    while num_same< batch_size//2 and i< len(same):
        if same[i][0] in name:
            i+=1
            continue
        name[ same[i][0] ]=1;   
        img1= extractFaceCo(format_filename( same[i][0], same[i][1] )  )
        img2= extractFaceCo(format_filename( same[i][0], same[i][2] )  )        
        while (len(img1)==0 or len(img2)==0): #fails to detect the face
            num_fails+=1
            if (i+ num_fails >= len(same) ):
                print("Did not find enough qualified images, please reduce the batch_size")
                not_enough=1
                break;
            
            img1= extractFaceCo(format_filename( same[-num_fails][0], same[-num_fails][1] )  )
            img2= extractFaceCo(format_filename( same[-num_fails][0], same[-num_fails][2] )  )
            
        if not_enough:
            break;
        num_same+=1    
        ident_in.append( img1.reshape(height, width, -1) )
        ident_out.append( img2.reshape(height, width, -1)  )
        ident_y.append(1);
        
    i=0    
    num_fails=0;
    num_diff=0
    not_enough=0
    while num_same+ num_diff< batch_size and i< len(diff):
        if diff[i][0] in name:
            i+=1
            continue
        name[ diff[i][0] ]=1
            
        img1= extractFaceCo(format_filename( diff[i][0], diff[i][1] )  )
        img2= extractFaceCo(format_filename( diff[i][2], diff[i][3] )  )      
        while (len(img1)==0 or len(img2)==0): #fails to detect the face
            num_fails+=1
            if (i+ num_fails >= len(same) ):
                print("Did not find enough qualified images, please reduce the batch_size")
                not_enough=1;
                break;
            
            img1= extractFaceCo(format_filename( diff[-num_fails][0], diff[-num_fails][1] )  )
            img2= extractFaceCo(format_filename( diff[-num_fails][2], diff[-num_fails][3] )  )
            
        if not_enough:
            break;
        num_diff+=1;
        ident_in.append( img1.reshape(height, width, -1) )
        ident_out.append( img2.reshape(height, width, -1)  )
        ident_y.append(0);
        
        
        
        


In [9]:
train_X, train_y= get_batch(1000, train_same, train_diff);
print("start to work on validation")
vali_X, vali_y= get_batch(200, vali_same, vali_diff);
print("start to work on test")
test_X, test_y= get_batch(20, test_same, test_diff);

../lfw-deepfunneled/George_P_Bush/George_P_Bush_0001.jpg
../lfw-deepfunneled/Juan_Pablo_Montoya/Juan_Pablo_Montoya_0005.jpg
../lfw-deepfunneled/Kate_Capshaw/Kate_Capshaw_0002.jpg
../lfw-deepfunneled/Ken_Watanabe/Ken_Watanabe_0001.jpg
../lfw-deepfunneled/Kobe_Bryant/Kobe_Bryant_0001.jpg
../lfw-deepfunneled/Budd_Schulberg/Budd_Schulberg_0001.jpg
../lfw-deepfunneled/Keith_Brown/Keith_Brown_0001.jpg
../lfw-deepfunneled/Clive_Lloyd/Clive_Lloyd_0001.jpg
../lfw-deepfunneled/Roy_Halladay/Roy_Halladay_0001.jpg
../lfw-deepfunneled/Hernan_Diaz/Hernan_Diaz_0001.jpg
../lfw-deepfunneled/Dereck_Whittenburg/Dereck_Whittenburg_0001.jpg
../lfw-deepfunneled/Julio_Rossi/Julio_Rossi_0001.jpg
../lfw-deepfunneled/Mickey_Rooney/Mickey_Rooney_0001.jpg
../lfw-deepfunneled/Irwan_Fadzi_Idris/Irwan_Fadzi_Idris_0001.jpg
../lfw-deepfunneled/Irwan_Fadzi_Idris/Irwan_Fadzi_Idris_0001.jpg
../lfw-deepfunneled/Pele/Pele_0001.jpg
../lfw-deepfunneled/Shingo_Katayama/Shingo_Katayama_0001.jpg
start to work on validation
start

In [12]:
# GRADED FUNCTION: triplet_loss

def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = K.sum( K.square( anchor- positive ), axis= -1 )
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = K.sum( K.square( anchor- negative ), axis= -1 )
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss =  pos_dist- neg_dist+ alpha
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = K.sum( K.maximum( basic_loss, 0.0 ) )
    ### END CODE HERE ###    
    return loss

FRmodel = faceRecoModel( [3, 96, 96] )
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [13]:
#use abs of different
EPSILON=0.0001
class FaceVerification:
    def __init__(self):
        
        self.encoding=[]
        self.same_person=[]
        self.score=0
        self.encoding_vali=[]
        
        self.data_base= {}
        
        
    def encodingImages(self, X, y): #list of 2 4 dimension array [batch, h, w, channels]
        self.encoding.clear()
        self.same_person.clear()
        for i in range( len( y ) ): #X[0] and X[1] form image pairs
            img1= X[0][i, :, :, :]  #(height, width, channel)
            img2= X[1][i, :, :, :]  #(height, width, channel)
            #encoding1= img_to_encoding( img1, FRmodel ).ravel()
            #encoding2= img_to_encoding( img2, FRmodel).ravel()
            #diff= (encoding1- encoding2)**2/ max(encoding1+ encoding2, EPSILON)
            
            
            diff= np.abs( img_to_encoding( img1, FRmodel ).ravel()- img_to_encoding( img2, FRmodel).ravel() )
            self.encoding.append( diff )  #list of 1d array
            self.same_person.append( y[i] )  #list of numbers 
            
    def encodingVali(self, X):
        self.encoding_vali.clear()
        for i in range( len( X[0] ) ): #X[0] and X[1] form image pairs
            img1= X[0][i, :, :, :]  #(height, width, channel)
            img2= X[1][i, :, :, :]  #(height, width, channel)
            #encoding1= img_to_encoding( img1, FRmodel ).ravel()
            #encoding2= img_to_encoding( img2, FRmodel).ravel()
            #diff= (encoding1- encoding2)**2/ max(encoding1+ encoding2, EPSILON) #Chi^2 distance
            diff= np.abs( img_to_encoding( img1, FRmodel  ).ravel()- img_to_encoding( img2, FRmodel  ).ravel() )
            self.encoding_vali.append( diff )  #list of 1d array
        
    
    def trainModelPredict(self, C): #Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
        model= LogisticRegression(penalty="l2", dual=False, tol=0.0001, solver='lbfgs', C=C )
        scores=cross_val_score(model, self.encoding, self.same_person, scoring="accuracy", cv = 5)
        self.score= scores.mean();
        
        model.fit(self.encoding, self.same_person)
        
        predicts=model.predict( self.encoding_vali)
        return predicts, self.score
            

In [36]:
#use Chi^2 distance (x-y)^2/(x+y)
EPSILON=0.0001
class FaceVerification:
    def __init__(self):
        
        self.encoding=[]
        self.same_person=[]
        self.score=0
        self.encoding_vali=[]
        
        self.data_base= {}
        
        
    def encodingImages(self, X, y): #list of 2 4 dimension array [batch, h, w, channels]
        self.encoding.clear()
        self.same_person.clear()
        for i in range( len( y ) ): #X[0] and X[1] form image pairs
            img1= X[0][i, :, :, :]  #(height, width, channel)
            img2= X[1][i, :, :, :]  #(height, width, channel)
            #encoding1= img_to_encoding( img1, FRmodel ).ravel()
            #encoding2= img_to_encoding( img2, FRmodel).ravel()
            #diff= (encoding1- encoding2)**2/ max(encoding1+ encoding2, EPSILON)
            
            
            diff= np.abs( img_to_encoding( img1, FRmodel ).ravel()- img_to_encoding( img2, FRmodel).ravel() )
            self.encoding.append( diff )  #list of 1d array
            self.same_person.append( y[i] )  #list of numbers 
            
    def encodingVali(self, X):
        self.encoding_vali.clear()
        for i in range( len( X[0] ) ): #X[0] and X[1] form image pairs
            img1= X[0][i, :, :, :]  #(height, width, channel)
            img2= X[1][i, :, :, :]  #(height, width, channel)
            #encoding1= img_to_encoding( img1, FRmodel ).ravel()
            #encoding2= img_to_encoding( img2, FRmodel).ravel()
            #diff= (encoding1- encoding2)**2/ max(encoding1+ encoding2, EPSILON) #Chi^2 distance
            diff= np.abs( img_to_encoding( img1, FRmodel  ).ravel()- img_to_encoding( img2, FRmodel  ).ravel() )
            self.encoding_vali.append( diff )  #list of 1d array
        
    
    def trainModelPredict(self, C): #Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
        model= LogisticRegression(penalty="l2", dual=False, tol=0.0001, solver='lbfgs', C=C )
        scores=cross_val_score(model, self.encoding, self.same_person, scoring="accuracy", cv = 5)
        self.score= scores.mean();
        
        model.fit(self.encoding, self.same_person)
        
        predicts=model.predict( self.encoding_vali)
        return predicts, self.score
 

In [37]:
face_veri=FaceVerification();
face_veri.encodingImages( train_X, train_y )
face_veri.encodingVali(vali_X);


In [39]:
scores=[ ]
for C in [0.01, 0.03, 0.1, 0.3,  1, 3, 10]:
    predicts, score= face_veri.trainModelPredict(C)  
    scores.append(score)
predictions, score= face_veri.trainModelPredict(3)


In [41]:
predictions, score= face_veri.trainModelPredict(3)

In [45]:
TN=sum( (vali_y== predictions)& (vali_y==0)  )/sum((vali_y==0))
TP=sum( (vali_y== predictions)& (vali_y==1)  )/sum((vali_y==1))
print(TP, TN, (TP+TN)/2 )

0.75 0.75 0.75


array([1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.])

0.85

In [25]:
class test:
  
    def __init__(self):
        self.a=3;
        self.b=5
        print(12)
    def aa(self):
        self.a=10
        self.b=[1,2,3] #this is only a local variable. use = carefully: means to declare a new variable.
    def cc(self):
        print(self.b)
        print(self.a)
       

In [27]:
e=test()
e.aa()
e.cc()

12
[1, 2, 3]
10


In [30]:
b=LogisticRegression(solver='lbfgs')
b.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [79]:
height, width=96, 96
input_= np.zeros( (1, 3, height, width) )
img= cv2.imread("database_images/Alexander_Lukashenko_0001.jpg")
print(img.shape)
img2=cv2.resize(img, (96, 96), interpolation = cv2.INTER_AREA)
print(img2.shape)
img3=np.rollaxis(img2, 2, 0 )
print(img3.shape)
input_[0, : , : , : ]= img3
a=img_to_encoding( img, FRmodel)
print(a.shape)

(250, 250, 3)
(96, 96, 3)
(3, 96, 96)
(1, 128)


In [127]:
c=img_to_encoding( cv2.imread("database_images/Alexander_Lukashenko_0001.jpg"), FRmodel).ravel()
print( type(c) )
print(c)
print(c.ravel() )

<class 'numpy.ndarray'>
[ 0.04595553  0.07246497  0.07501942  0.0937435   0.03919019  0.25245452
  0.05546704 -0.06767073  0.03881087 -0.06296956  0.02604692  0.07036097
  0.0349875  -0.10865398  0.06629271 -0.11094904 -0.0074172   0.09232423
 -0.09684844  0.01670366  0.01164339  0.03277656  0.02620441  0.21404396
  0.02198745 -0.18915516 -0.09475376 -0.09374249 -0.06817286  0.02285943
  0.13253497  0.03000964 -0.07266176  0.15166478  0.01422779  0.01063436
  0.07596198 -0.05064294 -0.02171043  0.06110191  0.1504443  -0.06104789
 -0.01450685 -0.11921036 -0.13727635 -0.2175961   0.02953147  0.00640469
 -0.00888163  0.11729448  0.08539972 -0.14602092  0.01938699  0.02699455
 -0.01152262  0.0814869  -0.03831456  0.08872418 -0.01724762 -0.02097518
 -0.12677157  0.09929045  0.02552805 -0.3351959   0.08579529  0.01957398
  0.1190557  -0.13662153 -0.26028576  0.08095471 -0.03529876  0.08136974
 -0.03445316  0.10672768  0.02060395 -0.0016467   0.03546085 -0.0476696
  0.01591405  0.01360815 -0.

In [81]:
a.shape
b=[]
b.append( a.ravel() )

In [83]:
b[0].shape

(128,)

In [77]:
image= cv2.imread("database_images/Alexander_Lukashenko_0001.jpg")
print( image.shape )
image = cv2.resize(image, (96, 96)) 
print( image.shape )
img = image[...,::-1]
print( img.shape )
img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
print( img.shape )

(250, 250, 3)
(96, 96, 3)
(96, 96, 3)
(3, 96, 96)


In [62]:
img= cv2.imread("database_images/Alexander_Lukashenko_0001.jpg")
print(img.shape)
img2=cv2.resize(img, (110, 110), interpolation = cv2.INTER_AREA)
print(img2.shape)
img3=np.rollaxis(img2, 2, 0 )
print(img3.shape)
a=img_to_encoding( img3, FRmodel)

(250, 250, 3)
(110, 110, 3)
(3, 110, 110)


ValueError: Error when checking input: expected input_1 to have shape (3, 96, 96) but got array with shape (110, 96, 96)

In [33]:
X=[ [i, i] if i< 50 else [-i, i-50]  for i in range(100)]
y=[ 1 if i< 50 else 0 for i in range(100)]
X_vali=[[1, 2], [-60, 0] ]

In [31]:
a= FaceVerification()
a.trainModel(X, y)

1.0